In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import sys
import gymnasium
sys.modules["gym"] = gymnasium
import pybullet as p
import matplotlib.pyplot as plt
from stable_baselines3 import SAC
# from robotiqGymEnv import robotiqGymEnv
from robotiqGymEnv import robotiqGymEnv
import numpy as np
import csv

def load_model(file_path):
    """
    Load the model from the given file path
    """
    return SAC.load(file_path)

def extract_data(env, model, obs, steps):
    """
    Extract data from the environment.
    """

    data = {
        "position_action": [],
        "angle_action": [],
        "gripper_position": [],
        "gripper_angle": [],
        "gripper_velocity": [],
        "gripper_angular_velocity": [],
        "target_position": [],
        "target_angle": [],
        "target_velocity": [],
        "target_angular_velocity": [],
        "closest_point": [],
        "contact_force": [],
        "rewards": [],
        "finger_angles": [[], [], []],  # finger1, finger2, finger3
        "finger_min_dists": [[], [], []]  # finger1, finger2, finger3
    }

    done = False
    j1_angle = 0

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, info = env.step(action)
        
        if rewards > 2:
            j1_angle = 100
            p.setJointMotorControl2(env._robotiq.robotiq_uid, 5, p.TORQUE_CONTROL, force=j1_angle)
        
        base_pos, base_orientation = p.getBasePositionAndOrientation(env._robotiq.robotiq_uid)
        target_pos, target_orientation = p.getBasePositionAndOrientation(env.blockUid)

        base_velocity, base_angular_velocity = p.getBaseVelocity(env._robotiq.robotiq_uid)
        target_velocity, target_angular_velocity = p.getBaseVelocity(env.blockUid)

        gripper_angle = p.getEulerFromQuaternion(base_orientation)
        target_angle = p.getEulerFromQuaternion(target_orientation)

        joint_indices = [1, 2, 3, 5, 6, 7, 9, 10, 11]
        joint_states = p.getJointStates(env._robotiq.robotiq_uid, joint_indices)
        
        for i in range(3):
            data["finger_angles"][i].append([joint_states[j][0] for j in range(i*3, (i+1)*3)])

        dist_indices = [1, 2, 3, 5, 6, 7, 9, 10, 11]
        min_dists = [p.getClosestPoints(env.blockUid, env._robotiq.robotiq_uid, 10, -1, i)[0][8] for i in dist_indices]

        for i in range(3):
            data["finger_min_dists"][i].append(min_dists[i*3:(i+1)*3])

        data["position_action"].append(action[0:3])
        data["angle_action"].append(action[3:6])
        data["gripper_position"].append(base_pos)
        data["gripper_angle"].append(gripper_angle)
        data["gripper_velocity"].append(base_velocity)
        data["gripper_angular_velocity"].append(base_angular_velocity)
        data["target_position"].append(target_pos)
        data["target_angle"].append(target_angle)
        data["target_velocity"].append(target_velocity)
        data["target_angular_velocity"].append(target_angular_velocity)
        data["closest_point"].append(p.getClosestPoints(env._robotiq.robotiq_uid, env.blockUid, 10, -1, -1)[0][8])
        data["contact_force"].append(env._contactinfo()[4])
        data["rewards"].append(rewards)
        
    return data

def plot_data(data, labels):
    """
    Plot the data using matplotlib
    """
    plt.figure()
    for d, label in zip(data, labels):
        plt.plot(d, label=label)
        plt.legend()

def save_to_csv(filename, data_dict):
    """
    Save data to CSV

    :param filename: Name of the file to save to
    :param data_dict: Dictionary containing labels as keys and data as values
    """

    # Get max length of rows for data arrays
    max_length = max(len(v) for v in data_dict.values())

    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write header
        headers = list(data_dict.keys())
        csv_writer.writerow(headers)

        # Write rows
        for i in range(max_length):
            row = [data_dict[header][i] if i < len(data_dict[header]) else "" for header in headers]
            csv_writer.writerow(row)

def main():
    """
    Main function to run the program
    
    """
    model_file = "models/20230316-03:42PM_SAC/best_model.zip"
    model = load_model(model_file)

    with robotiqGymEnv(records=False, renders=False) as env:
        obs = env.reset()
        steps = range(500) # define the number of steps here
        position_action_data, angle_action_data, gripper_position_data, gripper_angle_data, gripper_velocity_data, gripper_angular_velocity_data, target_position_data, target_angle_data, target_velocity_data, target_angular_velocity_data, closest_point_data, contact_force_data, rewards_data, finger1_angle_data, finger2_angle_data, finger3_angle_data, finger1_min_dist_data, finger2_min_dist_data, finger3_min_dist_data = extract_data(env, model, obs, steps)


    plot_data(zip(*position_action_data), ["x action", "y action", "z action"])
    # plot_data(zip(*angle_action_data), ["roll action", "pitch action", "yaw action"])
    plot_data(zip(*gripper_position_data), ["x position", "y position", "z position"])
    # plot_data(zip(*gripper_angle_data), ["gripper roll", "gripper pitch", "gripper yaw"])
    # plot_data(zip(*target_position_data), ["target x position", "target y position", "target z position"])
    # plot_data(zip(*target_angle_data), ["target roll", "target pitch", "target yaw"])
    plot_data([closest_point_data], ["closest distance"])
    plot_data([contact_force_data], ["contact force"])
    # plot_data([rewards_data], ["rewards"])
    plot_data([finger1_angle_data], ["finger1_angle_data"])
    plot_data([finger2_angle_data], ["finger2_angle_data"])
    plot_data([finger3_angle_data], ["finger3_angle_data"])
    plot_data([finger2_min_dist_data], ["finger2_min_dist_data"])

    data_dict = {
        "x action": [data[0] for data in position_action_data],
        "y action": [data[1] for data in position_action_data],
        "z action": [data[2] for data in position_action_data],
        "roll action": [data[0] for data in angle_action_data],
        "pitch action": [data[1] for data in angle_action_data],
        "yaw action": [data[2] for data in angle_action_data],
        "x gripper": [data[0] for data in gripper_position_data],
        "y gripper": [data[1] for data in gripper_position_data],
        "z gripper": [data[2] for data in gripper_position_data],
        "roll gripper": [data[0] for data in gripper_angle_data],
        "pitch gripper": [data[1] for data in gripper_angle_data],
        "yaw gripper": [data[2] for data in gripper_angle_data],
        "x velocity gripper": [data[0] for data in gripper_velocity_data],
        "y velocity gripper": [data[1] for data in gripper_velocity_data],
        "z velocity gripper": [data[2] for data in gripper_velocity_data],
        "x angular velocity gripper": [data[0] for data in gripper_angular_velocity_data],
        "y angular velocity gripper": [data[1] for data in gripper_angular_velocity_data],
        "z angular velocity gripper": [data[2] for data in gripper_angular_velocity_data],
        "x target": [data[0] for data in target_position_data],
        "y target": [data[1] for data in target_position_data],
        "z target": [data[2] for data in target_position_data],
        "roll target": [data[0] for data in target_angle_data],
        "pitch target": [data[1] for data in target_angle_data],
        "yaw target": [data[2] for data in target_angle_data],
        "x velocity target": [data[0] for data in target_velocity_data],
        "y velocity target": [data[1] for data in target_velocity_data],
        "z velocity target": [data[2] for data in target_velocity_data],
        "x angular velocity target": [data[0] for data in target_angular_velocity_data],
        "y angular velocity target": [data[1] for data in target_angular_velocity_data],
        "z angular velocity target": [data[2] for data in target_angular_velocity_data],
        "closest distance": closest_point_data,
        "contact force": contact_force_data,
        "rewards": rewards_data
    }

    save_to_csv("output_data30.csv", data_dict)

    gripper_position_data = np.array(gripper_position_data)
    gripper_final_position = np.mean(gripper_position_data[-100:, :], axis=0)
    target_position_data = np.array(target_position_data)
    target_final_position = np.mean(target_position_data[-100:, :], axis=0)
    # print("gripper final position: ", gripper_final_position)
    # print("target final position: ", target_final_position)
    D = 2628.9
    sat = 476.6
    robot_gripper_pose = [gripper_final_position[1]*1000, gripper_final_position[0]*1000, gripper_final_position[2]*1000]
    robot_sat_pose = [D - sat - target_final_position[1]*1000 - 100, - target_final_position[0]*1000, target_final_position[2]*1000 + 30]
    # print("robot gripper pose: ", robot_gripper_pose)
    # print("robot sat pose: ", robot_sat_pose)


if __name__ == "__main__":
    main()
